# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [219]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pymorphy2
import nltk
from nltk.metrics.distance import edit_distance
from nltk.stem import SnowballStemmer
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import csv
import random
from collections import Counter
from scipy.spatial.distance import cosine


In [46]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lisa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lisa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слве есть опечатка, если данное слово не содержится в списке `words`. 

In [3]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [4]:
words = []
with open('litw-win.txt') as fp:
    for line in fp:
        words.append(line.strip().split()[-1])
words[-5:]

['высокопревосходительства',
 'попреблагорассмотрительст',
 'попреблагорассмотрительствующемуся',
 'убегающих',
 'уменьшившейся']

In [5]:
sent = word_tokenize(text)
for i, word in enumerate(sent):
    if not word in words:
        sent[i] = min(words, key=lambda k: edit_distance(word, k))
' '.join(sent)

'с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий'

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [6]:
cond = '''Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. '''

In [7]:
stemmer = SnowballStemmer('russian')
sent = word_tokenize(cond)
res = []
for word in sent:
   res.append(stemmer.stem(word))
res[:5]

['счита', 'слов', 'из', 'файл', '`']

In [8]:
morph = pymorphy2.MorphAnalyzer()
sent = word_tokenize(cond)
res = []
for word in sent:
   res.append(morph.parse(word)[0].normalized.word)
res[:5]

['считать', 'слово', 'из', 'файл', '`']

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [9]:
cond = '''Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. '''
sents = sent_tokenize(cond)
sents

['Считайте слова из файла `litw-win.txt` и запишите их в список `words`.',
 'В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`.',
 'Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.']

In [10]:
cv = CountVectorizer()
cv.fit(sents)
sents_cv = cv.transform(sents)

In [11]:
sents_cv.toarray()

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]], dtype=int64)

In [12]:
cv.vocabulary_

{'считайте': 32,
 'слова': 24,
 'из': 12,
 'файла': 33,
 'litw': 0,
 'win': 2,
 'txt': 1,
 'запишите': 11,
 'их': 14,
 'список': 31,
 'words': 3,
 'заданном': 9,
 'предложении': 22,
 'исправьте': 13,
 'все': 5,
 'опечатки': 21,
 'заменив': 10,
 'опечатками': 20,
 'на': 16,
 'ближайшие': 4,
 'смысле': 27,
 'расстояния': 23,
 'левенштейна': 15,
 'ним': 18,
 'списка': 29,
 'что': 34,
 'слове': 25,
 'есть': 8,
 'опечатка': 19,
 'если': 7,
 'данное': 6,
 'слово': 26,
 'не': 17,
 'содержится': 28,
 'списке': 30}

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [85]:
with open('preprocessed_descriptions.csv') as fp:
    reader = csv.reader(fp)
    next(reader)
    words_all = []
    for row in reader:
        words_all.extend(word_tokenize(row[1]))
words = list(set(words_all))

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [14]:
for _ in range(5):
    w1 = words[random.randint(0, len(words)-1)]
    w2 = words[random.randint(0, len(words)-1)]
    print(w1, w2, edit_distance(w1, w2))

startedstill mobile 10
ja aromas 5
lunchessuppers awayyyyyyyy 14
cholent chiken 3
viscosity dewitt 7


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [42]:
def closest(word, k):
    ls = list(map(lambda x: [edit_distance(x, word), x], words))
    ls.sort()
    return [w[1] for w in ls[:k]]

In [43]:
closest("mildredr", 7)

['mildred', 'milder', 'milled', 'miller', 'milner', 'minded', 'molded']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [41]:
stemmer = SnowballStemmer('english')
wnlem = WordNetLemmatizer()
df = pd.DataFrame(list(map(lambda word: [word, stemmer.stem(word), wnlem.lemmatize(word)], words)), columns=['word', 'stemmed_word', 'normalized_word']).set_index('word')
df.head()

,stemmed_word,normalized_word
word,,
perth,perth,perth
dss,dss,ds
electrolytes,electrolyt,electrolyte
aka,aka,aka
additinal,additin,additinal


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [73]:
counter = Counter(words_all)
counter.most_common(10)

[('the', 40072),
 ('a', 34951),
 ('and', 30245),
 ('this', 26859),
 ('i', 24836),
 ('to', 23471),
 ('is', 20285),
 ('it', 19756),
 ('of', 18364),
 ('for', 15939)]

In [88]:
words_filt = list(filter(lambda x: x not in stopwords.words('english'), words_all))

In [92]:
(len(words_all) - len(words_filt))/len(words_all)

0.4567343213118679

In [93]:
counter = Counter(words_filt)
counter.most_common(10)

[('recipe', 14871),
 ('make', 6326),
 ('time', 5137),
 ('use', 4620),
 ('great', 4430),
 ('like', 4167),
 ('easy', 4152),
 ('one', 3872),
 ('made', 3810),
 ('good', 3791)]

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [179]:
descriptions = pd.read_csv('preprocessed_descriptions.csv')
descs = descriptions.sample(5)
descs.reset_index(drop=True, inplace=True)
descs

,name,preprocessed_descriptions
0,my way of steamed cabbage,i remember when i was growing up watching my g...
1,simple vanilla milkshake,simple and tasty i havent really tried this ...
2,pancake cupcakes with maple bacon buttercream ...,the cupcakes are based on a homemade pancake b...
3,apple squares or apple coffee cake,a neighbor brought these over as a thankyou gi...
4,buttermilk pumpkin waffles,from the blog of judicial peach


In [203]:
tv = TfidfVectorizer()
tv.fit(descs['preprocessed_descriptions'])
descs['vect'] = [i for i in tv.transform(descs['preprocessed_descriptions']).toarray()]
descs['vect']

0    [0.0, 0.08638831615212339, 0.0, 0.086388316152...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.24193021739697043,...
2    [0.0, 0.0, 0.13473625547007598, 0.0, 0.0, 0.09...
3    [0.14359060776268162, 0.0, 0.0, 0.0, 0.1158479...
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: vect, dtype: object

3.2 Вычислите близость между каждой парой рецептов, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [214]:
cos_dist = pd.DataFrame([[cosine(descs['vect'][i], descs['vect'][j]) for j in range(len(descs))] for i in range(len(descs))], columns=descs['name'], index=descs['name'])
cos_dist

name,my way of steamed cabbage,simple vanilla milkshake,pancake cupcakes with maple bacon buttercream frosting,apple squares or apple coffee cake,buttermilk pumpkin waffles
name,,,,,
my way of steamed cabbage,0.000000,0.884581,0.736059,0.956575,0.868206
simple vanilla milkshake,0.884581,0.000000,0.872714,1.000000,0.868348
pancake cupcakes with maple bacon buttercream frosting,0.736059,0.872714,0.000000,0.982645,0.889851
apple squares or apple coffee cake,0.956575,1.000000,0.982645,0.000000,1.000000
buttermilk pumpkin waffles,0.868206,0.868348,0.889851,1.000000,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат

In [216]:
cos_dist.min()

name
my way of   steamed cabbage                               0.0
simple vanilla milkshake                                  0.0
pancake cupcakes with maple bacon buttercream frosting    0.0
apple squares or apple coffee cake                        0.0
buttermilk pumpkin waffles                                0.0
dtype: float64

Наиболее похожие рецепты имеют косинусное растояние близкое к 0, на диагонали в дф стоят нули, так как это один и тот же рецепт. Остальные рецепты довольно разные, так как растояние очень близко к 1